In [9]:
import duckdb
import pandas as pd

# Path to your DuckDB database file
duckdb_file_path = '/workspace/databases/dev.duckdb'

# Use a context manager to handle the DuckDB connection
with duckdb.connect(duckdb_file_path) as con:
    # Query all schemas
    query_schemas = "SELECT schema_name FROM information_schema.schemata"
    schemas = con.execute(query_schemas).fetchdf()
    
    # Print the DataFrame of schemas
    print("Schemas:")
    print(schemas)
    
    # Query all tables in each schema
    query_tables = """
    SELECT table_schema, table_name 
    FROM information_schema.tables 
    WHERE table_schema IN (SELECT schema_name FROM information_schema.schemata)
    """
    tables = con.execute(query_tables).fetchdf()
    
    # Print the DataFrame of tables
    print("\nTables:")
    print(tables)

    # Check if the bronze schema exists
    schema_check_query = "SELECT schema_name FROM information_schema.schemata WHERE schema_name = 'bronze'"
    schema_exists = con.execute(schema_check_query).fetchone()
    if schema_exists:
        # Query the table created by dbt
        query = "SELECT * FROM bronze.customers LIMIT 5"
        result = con.execute(query).fetchdf()

        # Check if the result is not empty and print the DataFrame
        if not result.empty:
            print(result)
        else:
            print("No records found in the customers table.")
    else:
        print("The 'bronze' schema does not exist.")

Schemas:
           schema_name
0               bronze
1   information_schema
2                 main
3           pg_catalog
4               public
5   information_schema
6                 main
7           pg_catalog
8   information_schema
9                 main
10          pg_catalog

Tables:
  table_schema            table_name
0       bronze             customers
1       bronze                 dates
2       bronze              products
3       bronze         product_usage
4       bronze         subscriptions
5       bronze  support_interactions
6         main     test_duckdb_model
7       public     test_duckdb_model
   customer_id        name  age gender signup_date
0          101    John Doe   30      M  2020-01-15
1          102  Jane Smith   25      F  2019-06-20
2          103   Sam Brown   40      M  2021-03-10
3          104  Lisa White   35      F  2020-07-22
4          105   Tom Clark   28      M  2018-11-05


In [ ]:
# Use a context manager to handle the DuckDB connection
with duckdb.connect(duckdb_file_path) as con:
    # Query to delete tables
    query_delete_bronze = "DROP SCHEMA bronze CASCADE;"
    query_delete_silver = "DROP SCHEMA SILVER CASCADE;"
    query_delete_gold = "DROP SCHEMA GOLD CASCADE;"
    con.execute(query_delete_bronze)
    con.execute(query_delete_silver)
    con.execute(query_delete_gold)